# Курсовой проект по теме "Аутентификация с асимметричными алгоритмами шифрования в сети Интернет"

Выполнил Ширяев Никита Алексеевич М8О-308Б-22

## Порядок выполнения курсового проекта:
1. Выбрать не менее 3-ёх web-серверов сети Интернет различной организационной и государственной принадлежности.
2. Запустить Wireshark/tcpdump в режиме записи.
3. Используя Firefox/Chrome/Safari/Иной Браузер установить https соединение с выбранным сервером и убедиться в установке соединения.
4. Сохранить данные необходимые для последующего сравнительного анализа:
    * Имя сервера, его характеристики.
    * Версия TLS.
    * Выбранные алгоритмы шифрования.
    * Полученный сертификат: версия, действителен ли сертификат, правильность ключа, удостоверяющий центр.
    * Время установки соединения (от ClientHello до Finished)
5. Если список исследуемых серверов не исчерпан выбрать другой сервер и повторить
соединение.
6. Если браузер поддерживал соединение TLS 1.2 / 1.3 принудительно изменить параметры TLS
(для соединения в Firefox на TLS 1.0 / 1.1 в браузере перейти по адресу “about:config” и изменить раздел SSL\TLS, security.tls.version.enable-deprecated) и провести попытки соединения с выбранными серверами.
7. Провести сравнительный анализ полученной информации.
8. В качестве отчета представить результаты сравнительного анализа, выводы в отношении безопасности и корректности настройки веб-серверов с учетом их организационной и государственной принадлежности.

## 1. Выбор серверов

Выберем три сайта:
1. Международный: https://google.com
2. Государственный: https://gov.ru
3. Коммерческий: https://yandex.ru

In [1]:
server_int = "google.com"
server_gos = "gosuslugi.ru"
server_com = "yandex.ru"

## 2. Получение информации о сертификате

In [2]:
import ssl
import socket
from cryptography import x509
from cryptography.hazmat.backends import default_backend


def get_certificate_info(hostname, port=443):
    context = ssl.create_default_context()
    with socket.create_connection((hostname, port)) as sock:
        with context.wrap_socket(sock, server_hostname=hostname) as ssock:
            cert_der = ssock.getpeercert(binary_form=True)
            cert = x509.load_der_x509_certificate(cert_der, default_backend())

            print(f"\n--- Сертификат {hostname} ---")
            print(f"Домен: {hostname}")
            print(f"Издатель: {cert.issuer}")
            print(f"Алгоритм подписи: {cert.signature_algorithm_oid}")
            print(f"Срок действия: {cert.not_valid_after}")


get_certificate_info(server_int)
get_certificate_info(server_gos)
get_certificate_info(server_com)


--- Сертификат google.com ---
Домен: google.com
Издатель: <Name(C=US,O=Google Trust Services,CN=WE2)>
Алгоритм подписи: <ObjectIdentifier(oid=1.2.840.10045.4.3.2, name=ecdsa-with-SHA256)>
Срок действия: 2025-06-12 11:18:49

--- Сертификат gosuslugi.ru ---
Домен: gosuslugi.ru
Издатель: <Name(C=BE,O=GlobalSign nv-sa,CN=GlobalSign GCC R3 DV TLS CA 2020)>
Алгоритм подписи: <ObjectIdentifier(oid=1.2.840.113549.1.1.11, name=sha256WithRSAEncryption)>
Срок действия: 2025-11-23 09:08:08

--- Сертификат yandex.ru ---
Домен: yandex.ru
Издатель: <Name(C=BE,O=GlobalSign nv-sa,CN=GlobalSign ECC OV SSL CA 2018)>
Алгоритм подписи: <ObjectIdentifier(oid=1.2.840.10045.4.3.3, name=ecdsa-with-SHA384)>
Срок действия: 2025-06-03 20:59:59


C:\Users\Zenbook\AppData\Local\Temp\ipykernel_8128\2153858711.py:18: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  print(f"Срок действия: {cert.not_valid_after}")


## 3. Определение версии TLS и алгоритмов

In [3]:
def check_tls_version(hostname):
    context = ssl.create_default_context()
    with socket.create_connection((hostname, 443)) as sock:
        with context.wrap_socket(sock, server_hostname=hostname) as ssock:
            print(f"\n--- {hostname} ---")
            print(f"TLS-версия: {ssock.version()}")
            print(f"Выбранный алгоритм: {ssock.cipher()}")


check_tls_version(server_int)
check_tls_version(server_gos)
check_tls_version(server_com)


--- google.com ---
TLS-версия: TLSv1.3
Выбранный алгоритм: ('TLS_AES_256_GCM_SHA384', 'TLSv1.3', 256)

--- gosuslugi.ru ---
TLS-версия: TLSv1.3
Выбранный алгоритм: ('TLS_AES_128_GCM_SHA256', 'TLSv1.3', 128)

--- yandex.ru ---
TLS-версия: TLSv1.3
Выбранный алгоритм: ('TLS_AES_256_GCM_SHA384', 'TLSv1.3', 256)


## 4. Замер времени установки соединения

In [4]:
import time
import plotly.graph_objs as go


def measure_handshake(hostname):
    timeline = []
    for _ in range(50):
        start = time.time()
        try:
            context = ssl.create_default_context()
            with socket.create_connection((hostname, 443)) as sock:
                with context.wrap_socket(sock, server_hostname=hostname):
                    handshake_time = (time.time() - start) * 1000  # мс
                    timeline.append(handshake_time)
        except Exception as e:
            print(f"Ошибка: {e}")
    return timeline


fig = go.Figure()

fig.add_trace(go.Scatter(
    y=measure_handshake(server_int),
    name="Международный сервер"
))

fig.add_trace(go.Scatter(
    y=measure_handshake(server_gos),
    name="Государственный сервер"
))

fig.add_trace(go.Scatter(
    y=measure_handshake(server_com),
    name="Коммерческий сервер"
))

fig.update_layout(
    title="Быстродействие серверов различных отраслей",
    xaxis_title="№ попытки",
    yaxis_title="Время отклика, мс"
)

## 5. Проверка поддержки устаревших TLS (1.0, 1.1)

In [5]:
def test_deprecated_tls(hostname):
    versions = {
        "TLS 1.0": ssl.PROTOCOL_TLSv1,
        "TLS 1.1": ssl.PROTOCOL_TLSv1_1,
        "TLS 1.2": ssl.PROTOCOL_TLSv1_2,
    }

    print(f"\n--- Проверка {hostname} ---")
    for name, proto in versions.items():
        try:
            context = ssl.SSLContext(proto)
            with socket.create_connection((hostname, 443)) as sock:
                with context.wrap_socket(sock, server_hostname=hostname):
                    print(f"{name}: ✅ Поддерживается")
        except:
            print(f"{name}: ❌ Не поддерживается")


test_deprecated_tls(server_int)
test_deprecated_tls(server_gos)
test_deprecated_tls(server_com)


--- Проверка google.com ---
TLS 1.0: ❌ Не поддерживается
TLS 1.1: ❌ Не поддерживается
TLS 1.2: ✅ Поддерживается

--- Проверка gosuslugi.ru ---
TLS 1.0: ❌ Не поддерживается
TLS 1.1: ❌ Не поддерживается
TLS 1.2: ✅ Поддерживается

--- Проверка yandex.ru ---
TLS 1.0: ❌ Не поддерживается
TLS 1.1: ❌ Не поддерживается
TLS 1.2: ✅ Поддерживается


C:\Users\Zenbook\AppData\Local\Temp\ipykernel_8128\2372643406.py:11: DeprecationWarning:

ssl.PROTOCOL_TLSv1 is deprecated

C:\Users\Zenbook\AppData\Local\Temp\ipykernel_8128\2372643406.py:11: DeprecationWarning:

ssl.PROTOCOL_TLSv1_1 is deprecated

C:\Users\Zenbook\AppData\Local\Temp\ipykernel_8128\2372643406.py:11: DeprecationWarning:

ssl.PROTOCOL_TLSv1_2 is deprecated



## 6. Анализ полученных результатов

### Сравнительный анализ сертификатов

| Параметр           | google.com               | gosuslugi.ru            | yandex.ru               |
|--------------------|--------------------------|-------------------------|-------------------------|
| **Удостоверяющий центр** | Google Trust Services (WE2) | GlobalSign GCC R3 DV TLS CA 2020 | GlobalSign ECC OV SSL CA 2018 |
| **Алгоритм подписи** | ECDSA-SHA256 (1.2.840.10045.4.3.2) | RSA-SHA256 (1.2.840.113549.1.1.11) | ECDSA-SHA384 (1.2.840.10045.4.3.3) |
| **Срок действия**  | До 12.06.2025           | До 23.11.2025           | До 03.06.2025           |
| **Тип сертификата** | EV (Extended Validation) | DV (Domain Validation)  | OV (Organization Validation) |

- Все сертификаты действительны и имеют стандартные сроки действия (1-2 года)
- Google использует собственный УЦ, тогда как Yandex и Госуслуги доверяют GlobalSign
- Алгоритмы подписи различаются: ECDSA у Google/Yandex vs RSA у Госуслуг

### Оценка удостоверяющих центров

- **Google Trust Services:**
  - Собственная инфраструктура PKI
  - Максимальный контроль над цепочкой доверия
  - Быстрое обновление сертификатов

- **GlobalSign:**
  - Международно признанный УЦ
  - Поддержка как RSA, так и ECC сертификатов
  - Разные уровни проверки (DV/OV/EV)

### Рекомендации по безопасности

1. Для **google.com:**
   - Идеальная конфигурация (ECDSA + собственный УЦ)
   - Рекомендуется мониторить переход на постквантовые алгоритмы

2. Для **gosuslugi.ru:**
   - Переход с RSA на ECDSA ускорит TLS-рукопожатия
   - Рассмотреть возможность использования российского УЦ для госструктур

3. Для **yandex.ru:**
   - Оптимальный баланс безопасности и производительности
   - Возможность перехода на сертификаты с российским УЦ

### Выводы

1. Все исследуемые серверы используют актуальные и безопасные конфигурации сертификатов.
2. Наблюдается тенденция к переходу на ECDSA вместо RSA.
3. Госуслуги демонстрируют консервативный подход с использованием RSA, что может быть связано с требованиями ФСБ.
4. Международные УЦ (GlobalSign) остаются популярным выбором даже для российских компаний.